In [1]:
# !ls -hl|grep csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from time import time, ctime
from sklearn.metrics import accuracy_score, classification_report, classification, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from helper import plot_confusion_matrix, plot_confusion_matrix2
dim=lambda *x: [i.shape for i in x]

import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
df = pd.read_csv('./preprocessed.csv')
print df.shape

(49260, 748)
CPU times: user 2.86 s, sys: 232 ms, total: 3.09 s
Wall time: 3.16 s


In [3]:
X = df.iloc[:,2:].values
print X.shape
y = df.iloc[:,1].map({'male':1,'female':0}).values
print y.shape
df.head(1)

(49260, 746)
(49260,)


,username,fname.gender,username_split_predict,last,last_two,first,first2,nchar,vowels.pct,digits.pct,...,feature_721,feature_722,feature_723,feature_724,feature_725,feature_726,feature_727,feature_728,feature_729,feature_730
0,billion,male,unknow,n,on,b,bi,7,0.428571,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,14.0,7.0


In [4]:
%%time
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

for i in range(5):
    i=str(i)
    exec("labelencoder_X_{} = LabelEncoder()".format(i))
    exec("X[:, {}] = labelencoder_X_{}.fit_transform(X[:, {}])".format(i,i,i))

CPU times: user 144 ms, sys: 999 µs, total: 145 ms
Wall time: 144 ms


In [5]:
%%time
print X.shape
onehotencoder = OneHotEncoder(categorical_features = range(6))
X = onehotencoder.fit_transform(X).toarray()
print X.shape

(49260, 746)
(49260, 4505)
CPU times: user 1.31 s, sys: 485 ms, total: 1.79 s
Wall time: 1.79 s


In [6]:
%%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X = sc.fit_transform(X)
print X.shape

(49260, 4505)
CPU times: user 2.93 s, sys: 1.53 s, total: 4.46 s
Wall time: 4.45 s


https://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/

In [7]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier

import numpy as np

In [8]:
ctime()

'Sat Nov 11 00:44:46 2017'

In [ ]:
clf1 = AdaBoostClassifier()
clf2 = RandomForestClassifier(random_state=1, n_estimators=320, min_samples_split=4, n_jobs=-1)
clf3 = XGBClassifier(nthread=-1, max_depth=24,
                        min_child_weight=0.9, colsample_bytree=0.9,
                       scale_pos_weight= 0.9, reg_alpha=0.6)
clf4 = BaggingClassifier(n_jobs=-1, n_estimators=50)
clf5 = ExtraTreesClassifier(n_jobs=-1, bootstrap=True, n_estimators=100)

lr = LogisticRegression(solver='lbfgs', n_jobs=-1)

sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4, clf5], 
                          meta_classifier=lr)

print('5-fold cross validation:\n')

if __name__ == '__main__':
    stack1=[]
    for clf, label in zip([clf1, clf2, clf3, clf4, clf5,  sclf], 
                          ['AdaBoost', 
                           'Random Forest', 
                           'Xgboost',
                           'Bagging',
                           'ExtraTrees',
                           'Stacking']):

        scores = model_selection.cross_val_score(clf, X, y, 
#                                                  n_jobs=-1,
                                                  cv=5, scoring='accuracy')
        stack1.append([label, scores, scores.mean(), scores.std()])
        print("Accuracy: %0.4f (+/- %0.4f) [%s]" 
              % (scores.mean(), scores.std(), label))

5-fold cross validation:

Accuracy: 0.8237 (+/- 0.0008) [AdaBoost]
Accuracy: 0.8244 (+/- 0.0013) [Random Forest]
Accuracy: 0.8298 (+/- 0.0019) [Xgboost]
Accuracy: 0.8269 (+/- 0.0019) [Bagging]
Accuracy: 0.8268 (+/- 0.0012) [ExtraTrees]


In [ ]:
ctime()

In [ ]:
clf1 = AdaBoostClassifier()
clf2 = RandomForestClassifier(random_state=1, n_estimators=320, min_samples_split=4, n_jobs=-1)
clf3 = XGBClassifier(nthread=-1, max_depth=24,
                        min_child_weight=0.9, colsample_bytree=0.9,
                       scale_pos_weight= 0.9, reg_alpha=0.6)
clf4 = BaggingClassifier(n_jobs=-1, n_estimators=50)
clf5 = ExtraTreesClassifier(n_jobs=-1, bootstrap=True, n_estimators=100)

lr = LogisticRegression(solver='lbfgs', n_jobs=-1)

sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4, clf5], 
                          meta_classifier=lr, 
                         use_probas=True,
                          average_probas=False)

print('5-fold cross validation:\n')

if __name__ == '__main__':
    
    stack2 = []
    for clf, label in zip([clf1, clf2, clf3, clf4, clf5,  sclf], 
                          ['AdaBoost', 
                           'Random Forest', 
                           'Xgboost',
                           'Bagging',
                           'ExtraTrees',
                           'Stacking']):

        scores = model_selection.cross_val_score(clf, X, y, 
#                                                  n_jobs=-1,
                                                  cv=5, scoring='accuracy')
        stack2.append([label, scores, scores.mean(), scores.std()])
        print("Accuracy: %0.4f (+/- %0.4f) [%s]" 
              % (scores.mean(), scores.std(), label))

In [ ]:
ctime()

In [15]:
stack1

[['AdaBoost',
  array([ 0.82330255,  0.82511165,  0.8226756 ,  0.82369062,  0.8234697 ]),
  0.82365002369368789,
  0.00080507980493064015],
 ['Random Forest',
  array([ 0.82299807,  0.82592367,  0.82511165,  0.82521315,  0.82275911]),
  0.82440113197399723,
  0.0012764916941748496],
 ['Xgboost',
  array([ 0.82989952,  0.8317093 ,  0.8316078 ,  0.82917174,  0.82661659]),
  0.82980098897816001,
  0.001868275523513574],
 ['Bagging',
  array([ 0.82604283,  0.82998376,  0.82805522,  0.82531466,  0.8250939 ]),
  0.8268980724943058,
  0.0018630156210343507],
 ['ExtraTrees',
  array([ 0.82685477,  0.82835972,  0.82754771,  0.82643118,  0.82489087]),
  0.82681685010371453,
  0.0011639735131901646],
 ['Stacking',
  array([ 0.82746372,  0.82785221,  0.82846123,  0.82582217,  0.82570297]),
  0.82706045958704189,
  0.001107047276820309]]

In [16]:
stack2

[['AdaBoost',
  array([ 0.82330255,  0.82511165,  0.8226756 ,  0.82369062,  0.8234697 ]),
  0.82365002369368789,
  0.00080507980493064015],
 ['Random Forest',
  array([ 0.82299807,  0.82592367,  0.82511165,  0.82521315,  0.82275911]),
  0.82440113197399723,
  0.0012764916941748496],
 ['Xgboost',
  array([ 0.82989952,  0.8317093 ,  0.8316078 ,  0.82917174,  0.82661659]),
  0.82980098897816001,
  0.001868275523513574],
 ['Bagging',
  array([ 0.82523089,  0.82825822,  0.82886724,  0.82470564,  0.82377424]),
  0.8261672461092564,
  0.0020199828324036695],
 ['ExtraTrees',
  array([ 0.82726073,  0.82754771,  0.82825822,  0.8271417 ,  0.82560146]),
  0.82716196388000418,
  0.00087131062491349154],
 ['Stacking',
  array([ 0.83203085,  0.83353634,  0.83038977,  0.8271417 ,  0.8275302 ]),
  0.83012577140370336,
  0.002488882720539454]]